this notebook creates two maps that show the Hunza watershed with ESRI imagery
- first map overlays the algorithm calculated melt onset dates (MOD) for SIR pixels
- second map overlays the algorithm calculated melt onset dates (MOD) for GRD pixels

the pixel overlays are simply a scatter plot with rectangular markers and are not using EASE-grid projection

## Load in all the modules needed

In [1]:
%pylab notebook
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
# navigate to where scripts are saved
%cd /projects/brodzik/ipython_notebooks/melt_onset/scripts/
%ls

/projects/brodzik/ipython_notebooks/melt_onset/scripts
CETB_algorithms.py  CETB_analysis.py  CETB_read_functions.py  __pycache__/


In [3]:
# load the custom functions
from CETB_read_functions import read_Tb
from CETB_read_functions import coords
from CETB_read_functions import calc_DAV
from CETB_read_functions import find_UIB_cube_offset
from CETB_read_functions import grid_locations_of_UIB
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array
from CETB_analysis import MOD_array_year

## Specify inputs

In [4]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='UIB'  #make this the same syntax as cubefilenames and sub-directory
platform='AQUA'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sensor='AMSRE'  #'AMSRE', 'SSMI', etc.
channel='36V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
version='v1.3'
proj='N'

if sensor=='SSMI':
    provider='CSU' 
elif sensor=='AMSRE':
    provider='RSS'

cubeDir = '/work/PMESDR/CETB_v1.3/%s_%s/%s/cubes_%s/' % (platform, sensor, proj, region)    

# prefix filepath
prefix_GRD = 'CETB.cubefile.%s.%s_%s-%s-GRD-%s-%s' % (region, platform, sensor, channel, provider, version)
prefix_SIR = 'CETB.cubefile.%s.%s_%s-%s-SIR-%s-%s' % (region, platform, sensor, channel, provider, version)

# years for each sensor
if platform=='F13':
    Years = [2002,2003,2004,2005,2006,2007,2008,2009]
elif platform=='F14':
    Years=[2002,2003,2004,2005,2006,2007,2008]
elif platform=='F15':
    Years=[2002,2003,2004,2005,2006,2007,2008,2009,2010,2011]
elif platform=='AQUA':
    Years=[2003,2004,2005,2006,2007,2008,2009,2010,2011]

#Enter a site name for titles of plots
Site='Hunza'

In [5]:
# SPECIFY latitude and longitude in decimal degrees, need to choose lat/lon corners so that we will load
# in a rectangle of pixels within the corners of these coordinates
lat_start=35.9
lat_end=37.1
lon_start=74
lon_end=76

In [6]:
# get cube offset for finding row/col
# function is region specific
find_UIB_cube_offset(cubeDir, cubeType=None, verbose=True)

EASE2_N3.125km offsets for cubeType=36V-SIR
(Add these offsets to cube (row, col) to get row, col in full hemisphere)
offset row = 3120.000000, offset col = 4608.000000


(3120.000000031471, 4608.00000022618)

In [7]:
# get the GRD pixel IDs for the lat/lon rectangle chosen
rows_cols_GRD=coords(cubeDir, prefix_GRD, lat_start, lat_end, lon_start, lon_end)
rows_cols_GRD

(25, 33, 3, 8)

In [8]:
# row and col numbers of all the SIR pixels contained by the GRD pixels
# FIXME: this assumes SIR is 3.125 km data
rows_cols_env=[rows_cols_GRD[0]*8,rows_cols_GRD[1]*8, rows_cols_GRD[2]*8, rows_cols_GRD[3]*8]
rows_cols_env

[200, 264, 24, 64]

In [9]:
# load in SIR TB data
data_SIR=read_Tb(cubeDir, prefix_SIR, Years,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
CETB_SIR=data_SIR[0]   # 3-D Tb time-series array of TB
cal_date=data_SIR[1]    # 1-D array of dates, these will get passed to later functions
cal_year=data_SIR[2]    # 1-D array of years
cal_month=data_SIR[3]   # 1-D array of months

# load GRD Tb data
data_GRD=read_Tb(cubeDir, prefix_GRD, Years,rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
CETB_GRD=data_GRD[0]

In [10]:
#CETB_SIR.shape, CETB_SIR.dtype, np.nanmin(CETB_SIR), np.nanmax(CETB_SIR)

In [11]:
# calculate DAV for the Tb data that was imported
DAV_SIR=calc_DAV(CETB_SIR)
DAV_GRD=calc_DAV(CETB_GRD)

## get the latitudes and longitudes of each pixel for plotting

In [12]:
#Get lat/lon of each pixel in the subset of 64
# these get used for plotting over basemap
filename="%s%s.%4d.TB.nc" % (cubeDir, prefix_SIR, 2003)
data=Dataset(filename, "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_env[0]:rows_cols_env[1],rows_cols_env[2]:rows_cols_env[3]]

coordinates=coordinates.reshape(((rows_cols_env[1]-rows_cols_env[0])*(rows_cols_env[3]-rows_cols_env[2])),2)
pixel_lats=coordinates[:,0]
pixel_lons=coordinates[:,1]

pixel_lats

array([37.51086985, 37.48042702, 37.44997903, ..., 35.87361555,
       35.84317362, 35.81272617])

In [13]:
#Get lat/lon for GRD pixels
# similar to SIR above
filename="%s%s.%4d.TB.nc" % (cubeDir, prefix_GRD, 2003)
data=Dataset(filename, "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_GRD[0]:rows_cols_GRD[1],rows_cols_GRD[2]:rows_cols_GRD[3]]

coordinates=coordinates.reshape(((rows_cols_GRD[1]-rows_cols_GRD[0])*(rows_cols_GRD[3]-rows_cols_GRD[2])),2)
pixel_lats_GRD=coordinates[:,0]
pixel_lons_GRD=coordinates[:,1]

pixel_lats_GRD

array([37.37745373, 37.13370658, 36.8896261 , 36.64521097, 36.40045984,
       37.31533536, 37.07178516, 36.82789953, 36.58367717, 36.33911674,
       37.2521595 , 37.008809  , 36.76512093, 36.52109401, 36.27672694,
       37.18792871, 36.94478059, 36.70129274, 36.45746392, 36.21329284,
       37.12264554, 36.87970245, 36.63641747, 36.39278935, 36.14881684,
       37.05631258, 36.81357715, 36.57049763, 36.32707279, 36.08330139,
       36.98893244, 36.74640726, 36.50353575, 36.26031672, 36.01674895,
       36.92050778, 36.67819538, 36.43553441, 36.19252369, 35.94916202])

In [ ]:
# plot map of average MOD for year of interest                                                                      
def test_MOD_array_year(datadir, prefix, CETB_data, DAV, rows_cols, 
                        cal_date, year, window, count, DAV_threshold, Tb_threshold):                                                                                                            
        #GET MOD FRAME FOR SIR PIX                                                                                  
        # full matrix of MODs for every pixel in subset - SIR                                                       
        y_s=list(range(rows_cols[0],rows_cols[1]))                                                                  
        x_s=list(range(rows_cols[2],rows_cols[3]))                                                                  
        y_dims_list=list(range(len(CETB_data[0,:,0])))                                                              
        x_dims_list=list(range(len(CETB_data[0,0,:])))                                                              
                                                                                                                    
        melt_condition_met = (DAV>DAV_threshold) & (CETB_data[:,:,:]>Tb_threshold)  #the melt condition is met when
        # both the DAV and the Tb thresholds are exceeded                                                                    
        flag = melt_condition_met.astype(int)
        
        # Define a list of column_names with one for each array row, col
        # Populate each row with the flattened data for that date
        col_names = ["%s,%s" % (str(y),str(x)) for y in np.arange(rows_cols[0], rows_cols[1]) 
                     for x in np.arange(rows_cols[2], rows_cols[3])]
        newdata = zeros([flag.shape[0], flag.shape[1] * flag.shape[2]], dtype=flag.dtype)
        for d in np.arange(flag.shape[0]):
            newdata[d,] = flag[d, :, :].flatten()
        matrix = pd.DataFrame(data=newdata, columns=col_names)
                                        
        #matrix=pd.DataFrame() 
        #for i in y_dims_list:                                                                                       
        #        for j in x_dims_list:                                                                               
        #                column=pd.DataFrame(data=flag[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])                
        #                matrix=pd.concat([matrix,column],axis=1) 
        matrix=matrix.set_index(cal_date)  #index with dates (two rows per day)
        #print(matrix)
        
        #index with morning and evening offset (necessary for plotting)                                             
        m_index=matrix[0::2].index+pd.DateOffset(hours=9)                                                           
        e_index=matrix[1::2].index+pd.DateOffset(hours=18)                                                          
        t_index=np.append(m_index,e_index)                                                                          
        t_index=np.sort(t_index)                                                                                    
        matrix=matrix.set_index(t_index)                                                                            
        # MOD algorithm - calculate sum on a rolling window (window= no. of obs, 2 per day)                         
        matrix=matrix.rolling(window).sum()                                                                         
        matrix=matrix[matrix>=count]  # count= no. times thresholds are tripped for alg                             
        matrix=matrix.dropna(axis=0, how='all') # drop rows with all NaN                                            
 
        # gets a dataframe with one row for each pixel, 
        # one column for each year, MOD in DOY in each cell for that pixel and year                                                                                                      
        dates=np.empty(0)                                                                                           
        new_frame=pd.DataFrame()                                                                                    
                                                                                                                    
        for column in matrix:                                                                                       
                dates=np.append(dates,matrix[str(year)][column].first_valid_index())                                
                                                                                                                    
        dates_series=pd.Series(dates)                                                                               
        dates_series=dates_series.dt.dayofyear                                                                      
        new_frame=pd.concat([new_frame,dates_series], axis=1)                                                       
                                                                                                                    
        new_frame.columns=[str(year)]                                                                               
                                                                                                                    
        # get the average MOD for each pixel and make it an array for plotting                                      
        MOD=new_frame.mean(axis=1).values                                                                           
        MOD=np.ma.array(MOD)   # make it masked array                                                               
        MOD[MOD<0]=np.ma.masked   #convert any invalid MODs to masked                                               
                                                                                                                    
        return MOD  

## create arrays of MOD to be used for plotting

In [14]:
# get array of average MODs for SIR pixels for all the years loaded
window=10   # window for MOD algorithm, '10' would be 5 days (2 measurements per day)
count=3    # number of Tb/DAV exceedances to trigger MOD
DAV_threshold=40
Tb_threshold=252
year=2003

In [15]:
# sir MOD array - MOD will be in day of year (DOY)
MOD_DOY_array=MOD_array(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                        rows_cols_env, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array

KeyboardInterrupt: 

In [ ]:
MOD_DOY_array.shape

In [16]:
# get array of MOD for each pixel SIR - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
#year=2003

t = time.process_time()
MOD_DOY_array_year=MOD_array_year(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                                  rows_cols_env, cal_date, year, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array_year, elapsed_time

(masked_array(data=[100.0, 84.0, 132.0, ..., nan, nan, nan],
              mask=[False, False, False, ..., False, False, False],
        fill_value=1e+20), 3.701921065999997)

In [ ]:
# MOD of the GRD pixel - avg all years
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252

MOD_DOY_array_GRD=MOD_array(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                            rows_cols_GRD, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array_GRD

In [17]:
# get MOD for GRD pixel - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
#year=2003

t = time.process_time()

MOD_DOY_array_GRD_year=MOD_array_year(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                                      rows_cols_GRD, cal_date, year, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array_GRD_year, elapsed_time

(masked_array(data=[nan, nan, nan, nan, nan, nan, 132.0, 132.0, 143.0,
                    146.0, nan, 141.0, 143.0, 132.0, 141.0, 137.0, 139.0,
                    141.0, 153.0, 183.0, 130.0, 130.0, 153.0, nan, 155.0,
                    141.0, 139.0, nan, nan, nan, 130.0, 130.0, 141.0, nan,
                    nan, 128.0, 141.0, 132.0, nan, nan],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False],
        fill_value=1e+20), 0.32648495800000177)

In [ ]:
MOD_DOY_array_GRD.shape, MOD_DOY_array_GRD_year.shape

In [18]:
# Save these arrays to numpy zip
outfile = '/projects/brodzik/pmesdr_melt_onset/data/Hunza_MOD.npz'
#np.savez(outfile, 
#         MOD_DOY_array=MOD_DOY_array, 
#         MOD_DOY_array_year=MOD_DOY_array_year, 
#         MOD_DOY_array_GRD=MOD_DOY_array_GRD, 
#         MOD_DOY_array_GRD_year=MOD_DOY_array_GRD_year)
npzfile = np.load(outfile)
npzfile.files
# should return 4 variable names as a dict
orig_MOD_DOY_array = npzfile['MOD_DOY_array']
orig_MOD_DOY_array_year = npzfile['MOD_DOY_array_year']
orig_MOD_DOY_array_GRD = npzfile['MOD_DOY_array_GRD']
orig_MOD_DOY_array_GRD_year = npzfile['MOD_DOY_array_GRD_year']


In [21]:
orig_MOD_DOY_array_GRD_year

array([ nan,  nan,  nan,  nan,  nan,  nan, 132., 132., 143., 146.,  nan,
       141., 143., 132., 141., 137., 139., 141., 153., 183., 130., 130.,
       153.,  nan, 155., 141., 139.,  nan,  nan,  nan, 130., 130., 141.,
        nan,  nan, 128., 141., 132.,  nan,  nan])

In [22]:
MOD_DOY_array_GRD_year

masked_array(data=[nan, nan, nan, nan, nan, nan, 132.0, 132.0, 143.0,
                   146.0, nan, 141.0, 143.0, 132.0, 141.0, 137.0, 139.0,
                   141.0, 153.0, 183.0, 130.0, 130.0, 153.0, nan, 155.0,
                   141.0, 139.0, nan, nan, nan, 130.0, 130.0, 141.0, nan,
                   nan, 128.0, 141.0, 132.0, nan, nan],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False],
       fill_value=1e+20)

In [19]:
diff = MOD_DOY_array_GRD_year - orig_MOD_DOY_array_GRD_year
print(np.nanmin(diff), np.nanmax(diff))

0.0 0.0


In [20]:
orig_MOD_DOY_array_year

array([100.,  84., 132., ...,  nan,  nan,  nan])

In [23]:
MOD_DOY_array_year

masked_array(data=[100.0, 84.0, 132.0, ..., nan, nan, nan],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20)

In [24]:
diff = MOD_DOY_array_year - orig_MOD_DOY_array_year
print(np.nanmin(diff), np.nanmax(diff))

0.0 0.0


In [ ]:
t = time.process_time()
test=test_MOD_array_year(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                         rows_cols_GRD, cal_date, year, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
test, elapsed_time

In [ ]:
MOD_DOY_array_GRD_year

## create the maps

In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - SIR

# Choose plot type - 'year' for one year of interest, 'all' for average of all years
#plot_type='year'  
plot_type='all'

#year=2003


if plot_type=='all':
    array=MOD_DOY_array
    title = "%s-%s - MOD - (Avg DOY %4d-%4d)" % (platform, Site, Years[0], Years[-1])
elif plot_type=='year':
    array=MOD_DOY_array_year
    title = "%s-%s - MOD - (%4d DOY)" % (platform, Site, year)

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
# Why polar stereo???
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)

x,y=m(pixel_lons, pixel_lats)
m.scatter(x,y,c=array, s=30, marker='s',lw=0,cmap='RdBu', alpha=.6)
plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)


plt.title(title)
plt.show()


In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - GRD
# Choose plot type - 'year' for one year of interest, 'all' for average of all years
#plot_type='year'  
plot_type='all'

#year=2003


if plot_type=='all':
    array=MOD_DOY_array_GRD
    title = "%s-%s - MOD - (Avg DOY %4d-%4d)" % (platform, Site, Years[0], Years[-1])
elif plot_type=='year':
    array=MOD_DOY_array_GRD_year
    title = "%s-%s - MOD - (%4d DOY)" % (platform, Site, year)

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 
                'Hunza', color='red', linewidth=1)

x,y=m(pixel_lons_GRD, pixel_lats_GRD)
m.scatter(x,y,c=array, s=1200, marker='s',lw=0,cmap='RdBu', alpha=.6)

plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)
m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 
                'Hunza', color='red', linewidth=1)

plt.title(title)
plt.show()

In [ ]:
dav = np.diff(CETB_SIR, n=1, axis=0)

In [ ]:
CETB_SIR.shape, dav.shape

In [ ]:
np.nanmin(dav), np.nanmax(dav)

In [ ]:
dav = np.absolute(dav)

In [ ]:
np.nanmin(dav), np.nanmax(dav)

In [ ]:
dav[0, 0:5, 0:5]

In [ ]:
dav = np.insert(dav, [0], [0], axis=0)

In [ ]:
dav[0, 0:5, 0:5]

In [ ]:
dav[1, 0:5, 0:5]

In [ ]:
dav.shape


In [ ]:
df = pd.DataFrame()
print(df)

In [ ]:
#listOfStrings2 = ['Hi' for i in range(20)]
rows_cols

In [ ]:
rows_cols = rows_cols_GRD

In [ ]:
rows_cols


In [ ]:
mytest = [str(i)+','+str(j) for i in np.arange(1,4) for j in np.arange(1,3)]
mytest

In [ ]:
col_names = ["%s,%s" % (str(y),str(x)) for y in np.arange(rows_cols[0], rows_cols[1] + 1) for x in np.arange(rows_cols[2], rows_cols[3]+ 1)]
col_names

In [ ]:
len(col_names)

In [ ]:
df = pd.DataFrame(columns=col_names)

In [ ]:
df.columns

In [ ]:
myarr = np.reshape(np.array(np.arange(12)), (3,4))
myarr

In [ ]:
x = [True]
y = x.astype(int)
y

In [ ]:
CETB_GRD.shape[0]

In [ ]:
myarr = zeros([CETB_GRD.shape[0], CETB_GRD.shape[1] * CETB_GRD.shape[2]])
myarr.shape

In [ ]:
myarr[0,]

In [ ]:
CETB_GRD.shape

In [ ]:
CETB_GRD[1, :, :]

In [ ]:
CETB_GRD[1, :, :].flatten().shape

In [ ]:
df = pd.DataFrame(data=CETB_GRD[1, :, :])

In [ ]:
df

In [ ]:
df.values